In [1]:
import pandas as pd

In [3]:
# Let's make sure the kaggle.json file is present.
!ls -lha kaggle.json

-rw-r--r-- 1 root root 70 Nov 25 06:19 kaggle.json


In [4]:
!pip install -q kaggle

In [5]:
# The Kaggle API client expects this file to be in ~/.kaggle,
# so move it there.
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

# This permissions change avoids a warning on Kaggle tool startup.
!chmod 600 ~/.kaggle/kaggle.json

In [6]:
!pip install kaggle

In [7]:
!kaggle datasets download -d mlg-ulb/creditcardfraud

 76% 50.0M/66.0M [00:00<00:00, 139MB/s]
100% 66.0M/66.0M [00:00<00:00, 167MB/s]


In [8]:
!unzip creditcardfraud.zip -d Creditcardfraud

Archive:  creditcardfraud.zip
  inflating: Creditcardfraud/creditcard.csv  


In [9]:
df = pd.read_csv('/content/Creditcardfraud/creditcard.csv')
df.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
0,0.0,-1.359807,-0.072781,2.536347,1.378155,-0.338321,0.462388,0.239599,0.098698,0.363787,0.090794,-0.551600,-0.617801,-0.991390,-0.311169,1.468177,-0.470401,0.207971,0.025791,0.403993,0.251412,-0.018307,0.277838,-0.110474,0.066928,0.128539,-0.189115,0.133558,-0.021053,149.62,0
1,0.0,1.191857,0.266151,0.166480,0.448154,0.060018,-0.082361,-0.078803,0.085102,-0.255425,-0.166974,1.612727,1.065235,0.489095,-0.143772,0.635558,0.463917,-0.114805,-0.183361,-0.145783,-0.069083,-0.225775,-0.638672,0.101288,-0.339846,0.167170,0.125895,-0.008983,0.014724,2.69,0
2,1.0,-1.358354,-1.340163,1.773209,0.379780,-0.503198,1.800499,0.791461,0.247676,-1.514654,0.207643,0.624501,0.066084,0.717293,-0.165946,2.345865,-2.890083,1.109969,-0.121359,-2.261857,0.524980,0.247998,0.771679,0.909412,-0.689281,-0.327642,-0.139097,-0.055353,-0.059752,378.66,0
3,1.0,-0.966272,-0.185226,1.792993,-0.863291,-0.010309,1.247203,0.237609,0.377436,-1.387024,-0.054952,-0.226487,0.178228,0.507757,-0.287924,-0.631418,-1.059647,-0.684093,1.965775,-1.232622,-0.208038,-0.108300,0.005274,-0.190321,-1.175575,0.647376,-0.221929,0.062723,0.061458,123.50,0
4,2.0,-1.158233,0.877737,1.548718,0.403034,-0.407193,0.095921,0.592941,-0.270533,0.817739,0.753074,-0.822843,0.538196,1.345852,-1.119670,0.175121,-0.451449,-0.237033,-0.038195,0.803487,0.408542,-0.009431,0.798278,-0.137458,0.141267,-0.206010,0.502292,0.219422,0.215153,69.99,0


In [10]:
df.describe()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount,Class
count,284807.000000,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,2.848070e+05,284807.000000,284807.000000
mean,94813.859575,3.919560e-15,5.688174e-16,-8.769071e-15,2.782312e-15,-1.552563e-15,2.010663e-15,-1.694249e-15,-1.927028e-16,-3.137024e-15,1.768627e-15,9.170318e-16,-1.810658e-15,1.693438e-15,1.479045e-15,3.482336e-15,1.392007e-15,-7.528491e-16,4.328772e-16,9.049732e-16,5.085503e-16,1.537294e-16,7.959909e-16,5.367590e-16,4.458112e-15,1.453003e-15,1.699104e-15,-3.660161e-16,-1.206049e-16,88.349619,0.001727
std,47488.145955,1.958696e+00,1.651309e+00,1.516255e+00,1.415869e+00,1.380247e+00,1.332271e+00,1.237094e+00,1.194353e+00,1.098632e+00,1.088850e+00,1.020713e+00,9.992014e-01,9.952742e-01,9.585956e-01,9.153160e-01,8.762529e-01,8.493371e-01,8.381762e-01,8.140405e-01,7.709250e-01,7.345240e-01,7.257016e-01,6.244603e-01,6.056471e-01,5.212781e-01,4.822270e-01,4.036325e-01,3.300833e-01,250.120109,0.041527
min,0.000000,-5.640751e+01,-7.271573e+01,-4.832559e+01,-5.683171e+00,-1.137433e+02,-2.616051e+01,-4.355724e+01,-7.321672e+01,-1.343407e+01,-2.458826e+01,-4.797473e+00,-1.868371e+01,-5.791881e+00,-1.921433e+01,-4.498945e+00,-1.412985e+01,-2.516280e+01,-9.498746e+00,-7.213527e+00,-5.449772e+01,-3.483038e+01,-1.093314e+01,-4.480774e+01,-2.836627e+00,-1.029540e+01,-2.604551e+00,-2.256568e+01,-1.543008e+01,0.000000,0.000000
25%,54201.500000,-9.203734e-01,-5.985499e-01,-8.903648e-01,-8.486401e-01,-6.915971e-01,-7.682956e-01,-5.540759e-01,-2.086297e-01,-6.430976e-01,-5.354257e-01,-7.624942e-01,-4.055715e-01,-6.485393e-01,-4.255740e-01,-5.828843e-01,-4.680368e-01,-4.837483e-01,-4.988498e-01,-4.562989e-01,-2.117214e-01,-2.283949e-01,-5.423504e-01,-1.618463e-01,-3.545861e-01,-3.171451e-01,-3.269839e-01,-7.083953e-02,-5.295979e-02,5.600000,0.000000
50%,84692.000000,1.810880e-02,6.548556e-02,1.798463e-01,-1.984653e-02,-5.433583e-02,-2.741871e-01,4.010308e-02,2.235804e-02,-5.142873e-02,-9.291738e-02,-3.275735e-02,1.400326e-01,-1.356806e-02,5.060132e-02,4.807155e-02,6.641332e-02,-6.567575e-02,-3.636312e-03,3.734823e-03,-6.248109e-02,-2.945017e-02,6.781943e-03,-1.119293e-02,4.097606e-02,1.659350e-02,-5.213911e-02,1.342146e-03,1.124383e-02,22.000000,0.000000
75%,139320.500000,1.315642e+00,8.037239e-01,1.027196e+00,7.433413e-01,6.119264e-01,3.985649e-01,5.704361e-01,3.273459e-01,5.971390e-01,4.539234e-01,7.395934e-01,6.182380e-01,6.625050e-01,4.931498e-01,6.488208e-01,5.232963e-01,3.996750e-01,5.008067e-01,4.589494e-01,1.330408e-01,1.863772e-01,5.285536e-01,1.476421e-01,4.395266e-01,3.507156e-01,2.409522e-01,9.104512e-02,7.827995e-02,77.165000,0.000000
max,172792.000000,2.454930e+00,2.205773e+01,9.382558e+00,1.687534e+01,3.480167e+01,7.330163e+01,1.205895e+02,2.000721e+01,1.559499e+01,2.374514e+01,1.201891e+01,7.848392e+00,7.126883e+00,1.052677e+01,8.877742e+00,1.731511e+01,9.253526e+00,5.041069e+00,5.591971e+00,3.942090e+01,2.720284e+01,1.050309e+01,2.252841e+01,4.584549e+00,7.519589e+00,3.517346e+00,3.161220e+01,3.384781e+01,25691.160000,1.000000


In [12]:
df['Class'].value_counts()

0    284315
1       492
Name: Class, dtype: int64

In [13]:
X = df.iloc[:, 0:30]
y = df.iloc[:, 30]

In [14]:
y.head()

0    0
1    0
2    0
3    0
4    0
Name: Class, dtype: int64

In [15]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y, random_state = 0, test_size = 0.33)

In [16]:
X_train.head()

,Time,V1,V2,V3,V4,V5,V6,V7,V8,V9,V10,V11,V12,V13,V14,V15,V16,V17,V18,V19,V20,V21,V22,V23,V24,V25,V26,V27,V28,Amount
22827,32448.0,-0.544234,-0.448641,0.657485,-3.670843,1.281379,3.493796,-0.992897,1.001550,-2.289931,0.899774,-0.490759,-1.402153,0.405993,-0.542794,0.452350,0.315830,-0.344234,0.836372,-0.592480,-0.043908,-0.112491,-0.040475,-0.327215,0.981274,0.578130,-0.154278,0.313498,0.122708,8.40
104281,68996.0,1.099397,-0.201076,0.886129,0.406299,-0.747247,-0.115210,-0.479978,0.092256,0.166606,0.025720,1.169334,1.090135,0.760660,0.046660,0.668173,0.929527,-0.973930,0.543510,0.019095,0.088499,0.064920,0.078674,-0.044063,0.058322,0.165440,0.386638,-0.017080,0.023510,58.88
55205,46834.0,1.062558,0.145901,1.531429,2.608796,-0.789044,0.463536,-0.672808,0.330517,-0.089507,0.669475,0.674933,0.642812,-0.444117,-0.135842,-0.959016,0.837336,-0.642885,0.345980,-0.552866,-0.157424,-0.033093,-0.019463,0.028234,0.307752,0.250185,-0.060125,0.034603,0.026393,10.56
124479,77304.0,0.480369,-2.776255,-0.548201,-0.754582,-1.972704,-0.851128,-0.054217,-0.371061,-1.661668,1.176395,-0.999535,-1.517889,-0.977793,0.042723,-0.159776,-0.826886,0.952616,-0.020062,-0.091200,0.669371,0.100809,-0.496867,-0.555391,0.340263,0.437977,-0.114189,-0.096010,0.100662,559.12
75718,56219.0,-1.644622,0.999933,0.927572,1.088490,0.041400,0.861214,0.234369,0.563300,-0.476943,-0.010884,0.199895,0.092168,-1.197975,0.589714,-0.175797,-0.647336,0.375651,0.185812,1.397366,-0.275715,0.017683,-0.046211,-0.421555,-0.657974,0.443550,-0.152744,-0.560838,0.067824,61.90


In [17]:
from tensorflow.keras.layers import Dense, Flatten
from tensorflow.keras.models import Sequential
import tensorflow

In [18]:
n_inputs = 30

In [19]:
model = Sequential()
model.add(Dense(256, input_dim = n_inputs, activation= 'relu', kernel_initializer='he_uniform'))
model.add(Dense(1, activation = 'sigmoid'))

model.compile(optimizer = 'adam',
              loss = 'binary_crossentropy')
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 dense (Dense)               (None, 256)               7936      
                                                                 
 dense_1 (Dense)             (None, 1)                 257       
                                                                 
Total params: 8,193
Trainable params: 8,193
Non-trainable params: 0
_________________________________________________________________


In [20]:
model.fit(X_train, y_train, epochs = 50)

Epoch 1/50
5964/5964 [==============================] - 9s 1ms/step - loss: 56.4997
Epoch 2/50
5964/5964 [==============================] - 8s 1ms/step - loss: 30.3867
Epoch 3/50
5964/5964 [==============================] - 8s 1ms/step - loss: 20.4899
Epoch 4/50
5964/5964 [==============================] - 8s 1ms/step - loss: 17.4170
Epoch 5/50
5964/5964 [==============================] - 8s 1ms/step - loss: 16.3949
Epoch 6/50
5964/5964 [==============================] - 8s 1ms/step - loss: 12.9017
Epoch 7/50
5964/5964 [==============================] - 8s 1ms/step - loss: 11.4606
Epoch 8/50
5964/5964 [==============================] - 8s 1ms/step - loss: 12.7486
Epoch 9/50
5964/5964 [==============================] - 8s 1ms/step - loss: 11.9499
Epoch 10/50
5964/5964 [==============================] - 8s 1ms/step - loss: 10.8628
Epoch 11/50
5964/5964 [==============================] - 8s 1ms/step - loss: 10.4844
Epoch 12/50
5964/5964 [==============================] - 8s 1ms/step - los

In [21]:
from sklearn.metrics import roc_auc_score

y_pred = model.predict(X_test)

print(roc_auc_score(y_test,y_pred))

0.9046165536706438


In [22]:
y_train.value_counts()

0    190490
1       330
Name: Class, dtype: int64

# **Weight assigning**



In [23]:
weights_assigned={0:1, 1:570}

In [24]:
model = Sequential()
model.add(Dense(256, input_dim = n_inputs, activation = 'relu', kernel_initializer = 'he_uniform'))
model.add(Dense(1, activation = 'sigmoid'))
model.compile(loss = 'binary_crossentropy',
              optimizer = 'adam')

model.fit(X_train, y_train, epochs = 50, class_weight=weights_assigned)

Epoch 1/50
5964/5964 [==============================] - 9s 1ms/step - loss: 4890.0801
Epoch 2/50
5964/5964 [==============================] - 9s 2ms/step - loss: 3518.9890
Epoch 3/50
5964/5964 [==============================] - 9s 2ms/step - loss: 2671.9954
Epoch 4/50
5964/5964 [==============================] - 9s 1ms/step - loss: 3035.6304
Epoch 5/50
5964/5964 [==============================] - 9s 1ms/step - loss: 2624.1641
Epoch 6/50
5964/5964 [==============================] - 9s 2ms/step - loss: 2363.5930
Epoch 7/50
5964/5964 [==============================] - 9s 2ms/step - loss: 2490.6570
Epoch 8/50
5964/5964 [==============================] - 9s 1ms/step - loss: 2574.8215
Epoch 9/50
5964/5964 [==============================] - 9s 2ms/step - loss: 2359.8691
Epoch 10/50
5964/5964 [==============================] - 9s 1ms/step - loss: 2510.9771
Epoch 11/50
5964/5964 [==============================] - 9s 2ms/step - loss: 2410.9514
Epoch 12/50
5964/5964 [=============================

In [25]:
y_pred=model.predict(X_test)

from sklearn.metrics import roc_auc_score
print(roc_auc_score(y_test,y_pred))

0.9184252926876606


**We can see that the accuracy has been developed after weight assigning. This way we can improve the accuracy even if the dataset is imbalanced.**